## Fetching weather data from openweathermap.org

It's possible to get data for the past 5 days with an unpaid account, but getting data any further back costs money.
As of now I only have data more than 5 days back for one station: Axis-Brightwood. I'm considering buying historical data for that one site, but for now the following code will fetch recent data for all sites at which I'm actively collecting data.
I need to manually run this notebook once a day until I get around to copy-pasting this into a script and automating that task.

In [1]:
import json, os, requests, time
from datetime import datetime, timedelta
import pymysql
import sqlalchemy as SQL
from urllib.parse import quote_plus as QP
HOME = os.path.expanduser('~')
WEATHER_DIR = os.path.join(HOME, 'Data', 'Storage', 'AlertWF', 'weather')

if not os.path.exists(WEATHER_DIR):
    os.makedirs(WEATHER_DIR)
assert os.path.isdir(WEATHER_DIR)

secrets_path = os.path.join(HOME, 'Documents', 'secrets.json')
with open(secrets_path,'r') as secrets_file:
    secrets = json.load(secrets_file)
    API_KEY = secrets['openweathermap.org']['api_key']
    _sql_secrets = secrets['mysql']
    sql_passwd = _sql_secrets['PASSWD']
    sql_usr = _sql_secrets['USER']
    sql_host = _sql_secrets['HOST']
    sql_port = _sql_secrets['PORT']
    sql_db = _sql_secrets['DB']

WEATHER_API = 'https://api.openweathermap.org/data/2.5/onecall/timemachine'
SQL_URL = f"mysql+pymysql://{sql_usr}:{QP(sql_passwd)}@{sql_host}:{sql_port}/{sql_db}"
SQL_ENGINE = SQL.create_engine(SQL_URL)
%load_ext sql
%sql $SQL_URL

In [2]:
def get_hourly_weather(lon:float, lat:float, date:datetime) -> dict:
    dt = int(date.timestamp())
    prm = dict(lon=lon,lat=lat,dt=dt,units='metric',appid=API_KEY)
    result = requests.get(WEATHER_API, params=prm)
    assert result.status_code == 200, \
        f'status code: {result.status_code}\nURL: {result.url}'
    time.sleep(1) ## Rate-limited API
    return result.json()

def days_ago(n: int) -> datetime:
    d = datetime.date(datetime.today())-timedelta(days=n)
    dt = datetime.fromordinal(d.toordinal())
    return dt

In [3]:
query = 'SELECT DISTINCT station, lon, lat FROM stations st INNER JOIN images im ON st.id=im.station;'

with SQL_ENGINE.connect().execution_options(autocommit=True) as conn:
    results = conn.execute(query)
    for station, lon, lat in results.fetchall():
        #for n in range(5):
        for n in [1]:
            dt = days_ago(n)
            weather = get_hourly_weather(lon, lat, dt)
            fname = f"{station}-{dt.strftime('%Y-%m-%d')}.json"
            path = os.path.join(WEATHER_DIR, fname)
            print('  Saving:', path)
            with open(path, 'w') as data_file:
                json.dump(weather, data_file)

print('Done!')        

  Saving: /home/jmp/Data/Storage/AlertWF/weather/Axis-Alpine-2021-03-16.json
  Saving: /home/jmp/Data/Storage/AlertWF/weather/Axis-Brightwood-2021-03-16.json
  Saving: /home/jmp/Data/Storage/AlertWF/weather/Axis-DeerHorn2-2021-03-16.json
  Saving: /home/jmp/Data/Storage/AlertWF/weather/Axis-FayRanch2-2021-03-16.json
  Saving: /home/jmp/Data/Storage/AlertWF/weather/Axis-GrizzlyPeakLookout2-2021-03-16.json
  Saving: /home/jmp/Data/Storage/AlertWF/weather/Axis-HolidayLake2-2021-03-16.json
  Saving: /home/jmp/Data/Storage/AlertWF/weather/Axis-Indiana-2021-03-16.json
  Saving: /home/jmp/Data/Storage/AlertWF/weather/Axis-Steens-2021-03-16.json
  Saving: /home/jmp/Data/Storage/AlertWF/weather/Axis-Strawberry2-2021-03-16.json
  Saving: /home/jmp/Data/Storage/AlertWF/weather/Axis-WestMtnUtah-2021-03-16.json
  Saving: /home/jmp/Data/Storage/AlertWF/weather/Axis-WhitneyPortal2-2021-03-16.json
Done!


In [4]:
requests.get?

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary or bytes to be sent in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      ~/.local/lib/python3.8/site-packages/requests/api.py
Type:      function
